# Guy Ben Dor , Yotam Hazan

Functions

In [ ]:
from typing import List, Any
import mediapipe as mp
import cv2
import time
import xlsxwriter
import numpy as np
from scipy.signal import savgol_filter
from tkinter import filedialog
import os
import pyautogui
import matplotlib.pyplot as plt

def speed_checker(a_1, a_3, t_1, t_3) -> object:
    if (t_3 - t_1) == 0:
        v_2 = 0
    else:
        v_2 = (a_3 - a_1) / (t_3 - t_1)
    return abs(v_2)

def acceleration_checker(v_1, v_3, t_1, t_3) :
    if (t_3 - t_1) == 0:
        v_2 = 0
    else:
        v_2 = (v_3 - v_1) / (t_3 - t_1)

    return v_2

def time_save(b,c,d,e):
    a = b
    b = c
    c = d
    d = e
    e = 0.033+ d

    return [a,b,c,d,e]


def loc_save(b, c, d, e, new, correct):
    if correct == 1:
        a = b
        b = c
        c = d
        d = round((d+new)/2,3)
        e = new

    else:
        a = b
        b = c
        c = d
        d = e
        e = new
    return [a, b, c, d, e]


def angle_calculater(landmarks,mp_pose):
    shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    knee_left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    knee_right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    ankle_left = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    ankle_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    ear_left = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
    ear_right = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                 landmarks[mp_pose.PoseLandmark.NOSE.value].y]
    center_shoulder = [abs((landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x + landmarks[
        mp_pose.PoseLandmark.LEFT_SHOULDER.value].x) / 2),
                       abs((landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y  +landmarks[
                           mp_pose.PoseLandmark.LEFT_SHOULDER.value].y) / 2)]
    hand_index_left = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
    hand_index_right = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
    leg_index_left = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    leg_index_right = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

    angle_elbow_L = calculate_angle(shoulder_left, elbow_left, wrist_left)
    angle_shoulder_L = calculate_angle(hip_left, shoulder_left, elbow_left)
    angle_elbow_R = calculate_angle(shoulder_right, elbow_right, wrist_right)
    angle_shoulder_R = calculate_angle(hip_right, shoulder_right, elbow_right)
    angle_elbow_to_l_shoulder = calculate_angle(shoulder_left, shoulder_right, elbow_right)
    angle_elbow_to_r_shoulder = calculate_angle(shoulder_right, shoulder_left, elbow_left)
    angle_hip_L = calculate_angle(knee_left, hip_left, shoulder_left)
    angle_hip_R = calculate_angle(knee_right, hip_right, shoulder_right)
    angle_inner_hip_L = calculate_angle(hip_right, hip_left, knee_left)
    angle_inner_hip_R = calculate_angle(hip_left, hip_right, knee_right)
    angle_knee_L = calculate_angle(hip_left, knee_left, ankle_left)
    angle_knee_R = calculate_angle(hip_right, knee_right, ankle_right)
    angle_L_skull = calculate_angle(shoulder_left, ear_left, center_shoulder)
    angle_R_skull = calculate_angle(shoulder_right, center_shoulder,ear_right)
    angle_l_hand = calculate_angle(elbow_left, wrist_left, hand_index_left)
    angle_r_hand = calculate_angle(elbow_right, wrist_right, hand_index_right)
    angle_l_leg = calculate_angle(knee_left, ankle_left, leg_index_left)
    angle_r_leg = calculate_angle(knee_right, ankle_right, leg_index_right)
    angle_wirst_to_r_hip = calculate_angle(wrist_right, shoulder_right, hip_right)
    angle_wirst_to_l_hip = calculate_angle(wrist_left, shoulder_left, hip_left)
    # print(angle_R_skull) canhge it back

    return Angels(angle_elbow_R, angle_shoulder_R, angle_elbow_L, angle_shoulder_L,
                  angle_elbow_to_l_shoulder, angle_elbow_to_r_shoulder, angle_hip_L,
                  angle_hip_R, angle_inner_hip_L, angle_inner_hip_R, angle_knee_L, angle_knee_R,
                  angle_L_skull, angle_R_skull, angle_l_hand, angle_r_hand, angle_l_leg, angle_r_leg,
                  angle_wirst_to_r_hip, angle_wirst_to_l_hip)



def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# class Karate_position:
#     def __init__(self,punch):

class Angels:
    def __init__(self, angle_elbow_right, angle_shoulder_right, angle_elbow_left, angle_shoulder_left,
                 angle_elbow_to_l_shoulder, angle_elbow_to_r_shoulder, angle_hip_l, angle_hip_r, angle_inner_hip_l,
                 angle_inner_hip_r, angle_knee_l, angle_knee_r, angle_l_skull, angle_r_skull, angle_l_hand,
                 angle_r_hand, angle_l_leg, angle_r_leg, angle_wirst_to_r_hip,angle_wirst_to_l_hip):
        self.angle_elbow_right = angle_elbow_right
        self.angle_shoulder_right = angle_shoulder_right
        self.angle_elbow_left = angle_elbow_left
        self.angle_shoulder_left = angle_shoulder_left
        self.angle_elbow_to_l_shoulder = angle_elbow_to_l_shoulder
        self.angle_elbow_to_r_shoulder = angle_elbow_to_r_shoulder
        self.angle_hip_r = angle_hip_r
        self.angle_hip_l = angle_hip_l
        self.angle_inner_hip_l = angle_inner_hip_l
        self.angle_inner_hip_r = angle_inner_hip_r
        self.angle_knee_l = angle_knee_l
        self.angle_knee_r = angle_knee_r
        self.angle_l_skull = angle_l_skull
        self.angle_r_skull = angle_r_skull
        self.angle_l_hand = angle_l_hand
        self.angle_r_hand = angle_r_hand
        self.angle_l_leg = angle_l_leg
        self.angle_r_leg = angle_r_leg
        self.angle_wirst_to_r_hip = angle_wirst_to_r_hip
        self.angle_wirst_to_l_hip = angle_wirst_to_l_hip

    # @property
    def pose_data(self):
        return [self.angle_elbow_right, self.angle_shoulder_right, self.angle_elbow_left,
                self.angle_shoulder_left, self.angle_elbow_to_l_shoulder, self.angle_elbow_to_r_shoulder,
                self.angle_hip_l, self.angle_hip_r, self.angle_inner_hip_l, self.angle_inner_hip_r, self.angle_knee_l,
                self.angle_knee_r, self.angle_l_skull,
                self.angle_r_skull, self.angle_l_hand, self.angle_r_hand,self.angle_l_leg,self.angle_r_leg,
                self.angle_wirst_to_r_hip,self.angle_wirst_to_l_hip]


def sava_speed(b, c, d, e, time, new, status):  # 0- time 1- pix 2- currect
    if status == 0:
        a = b
        b = c
        c = d
        d = e
        e = time
    elif status == 1:
        a = b
        b = c
        c = d
        d = round((new+d)/2, 3)
        e = round(new,3)
    else:
        a = b
        b = c
        c = d
        d = e
        e = round(new,3)

    return [a, b, c, d, e]


def shift_register(data,new_data):
    data[0] = data[1]
    data[1] = data[2]
    data[2] = data[3]
    data[3] = data[4]
    data[4] = data[5]
    data[5] = data[6]
    data[6] = data[7]
    data[7] = data[8]
    data[8] = data[9]
    data[9] = new_data

    return data

def shift_register_speed(data,new_data):
    data[0] = data[1]
    data[1] = data[2]
    data[2] = data[3]
    data[3] = data[4]
    data[4] = data[5]
    data[5] = data[6]
    data[6] = data[7]
    data[7] = data[8]
    data[8] = data[9]
    data[9] = new_data

    return data

def sava_acceleration(speed, time, new, status):  # 0- currect  2- time
    if status == 0:
        speed[0] = speed[1]
        speed[1] = speed[2]
        speed[2] = speed[3]
        speed[3] = speed[4]
        speed[4] = time
    else:
        speed[0] = speed[1]
        speed[1] = speed[2]
        speed[2] = speed[3]
        speed[3] = speed[4]
        speed[4] = round(new,3)
    return speed


def pose_checker_punch(position,movment_data,en_l_punch,en_r_punch):
    # ---------------- right punch ------------- side angle

    if ( 2>=movment_data[18][0] >=1):
        if 60 < position[1][3]  and 140 < position[0][3] < 180 and position[19][3]<60 and position[18][3]>60:
            stage = "right punch"
            print("right punch")
            return stage

        if 60 < position[1][3]  and 65 > position[0][3]  and position[18][3] > 60 and en_r_punch!=1:

            print('elbow')
            return  "right elbow"

    if ( movment_data[19][0] ==2):

        if 60 < position[3][3]  and 140 < position[2][3] < 180 and position[18][3]<60 and position[19][3]>60:
            stage = "left punch"
            print("left punch")
            return stage
        elif 60 < position[3][3]  and 65 > position[2][3]  and position[19][3] > 60 and en_l_punch!=1:

            print('left elbow')
            return  "left elbow"



def pose_checker_kick(position,movment_data,en_r_kick,en_l_kick):
    if ( movment_data[7][0] == 2 or  movment_data[9][0] == 2):
        if (position[7][3]<100 or position[9][3]>130) and position[6][3]> 125 and position[11][3]>110:
            print("right kick")
            stage = "right kick"

            return stage
        elif (position[7][3]<100 or position[9][3]>130) and position[6][3]> 125 and position[11][3]<100 and en_r_kick !=1:
            print("right knee")
            stage = "right knee"
            return stage
    if ( movment_data[6][0] == 2 or  movment_data[8][0] == 2):
        if (position[6][3]<100 or position[8][3]>130) and position[7][3]> 125 and position[10][3]>110:
            print("left kick")
            stage = "left kick"

            return stage
        elif (position[6][3]<100 or position[8][3]>130) and position[7][3]> 125 and position[10][3]<100 and en_l_kick !=1:
            print("left knee")
            stage = "left knee"
            return stage




def movment_data_cheker(i, loc, speed, acceleration, movment_data):

    if speed[4] >= speed[3] >= speed[2] \
            >= speed[1] >= speed[0] > 0 and \
            movment_data[0] == 0 and abs(loc[0] - loc[4]) > sum(loc)/8 and loc[4]>15:  # change and find something else \and loc[0] < 45 for a punch
        if (i == 9 or i==8 or i ==0 or (i == 1 and loc[4]<30)   or i==2 or i==3 or i == 19 or i == 18) and loc[0] > loc[3] :
            return movment_data
        elif (i == 6 or i==7  ) and loc[0] < loc[3]:
            return movment_data

        bmf = "start"
        status = 1
        angle_thredhold_in = (sum(loc[0:4]) / (len(loc)-1 ))
        if loc[0] < loc[3]:
            up = 1
        elif loc[0] > loc[3]:
            up = 0
        return [status, bmf,angle_thredhold_in, movment_data[3], up]
    elif movment_data[0] == 1 and acceleration[0] > 0 and acceleration[1] > 0 \
            and acceleration[2] > 0:
        bmf = "enter move"
        angle_thredhold_out = (sum(loc[3:5]) / (len(loc) - 3))
        status = 2
        return [status, bmf, movment_data[2],angle_thredhold_out, movment_data[4]]

    elif movment_data[0] == 2 and acceleration[2] < 0 and acceleration[3] < 0 \
            and acceleration[4] >= 0:
        if movment_data[4] == 1 and movment_data[3] > loc[4]:
            bmf = "return"
            status = 3
            return [status, bmf, movment_data[2],movment_data[3], movment_data[4]]
        elif movment_data[4] == 0 and movment_data[3] < loc[4]:
            bmf = "return"
            status = 3
            return [status, bmf, movment_data[2],movment_data[3], movment_data[4]]
    elif movment_data[0] == 3:
        if movment_data[4] == 1 and movment_data[2] > loc[2]:
            bmf = "finish"
            status = 0
            return [status, bmf, movment_data[2],movment_data[3], movment_data[4]]
        elif movment_data[4] == 0 and movment_data[2] < loc[2]:
            bmf = "finish"
            status = 0
            return [status, bmf, movment_data[2],movment_data[3], movment_data[4]]
    elif movment_data[1] == "finish":
        return [0, "null", 0, 0, 0]

    return movment_data


def live_section_teacher():
    cap = cv2.VideoCapture(0 , cv2.CAP_DSHOW)
    teacher_or_student = 'teacher'
    main_shift(cap,teacher_or_student)

def video_selecter_teacher():
    path = os.getcwd()
    selected_video = filedialog.askopenfilename(initialdir = path,title = "Select a File",filetypes = (("mp4 files", "*.mp4"),("all files","*.*")))
    cap = cv2.VideoCapture(selected_video)
    teacher_or_student = 'teacher'
    main_shift(cap,teacher_or_student)

def live_section_student():
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    teacher_or_student = 'student'
    main_shift(cap,teacher_or_student)

def main_shift(cap,teacher_or_student):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    stage_punch = "non"
    save_time = [0, 0, 0, 0, 0]
    cu = 0
    save_loc = []
    speed_save = []
    acceleration_save = []
    movement_data = []
    save_data = []
    row_moves = 1
    dn_angle = []
    dn_speed = []
    en_r_kick=0
    en_l_kick = 0
    en_r_punch = 0
    en_l_punch = 0
    en_r_punch =0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        row = 1
        if teacher_or_student == 'teacher':
            angle = xlsxwriter.Workbook('TeacherDataAngle.xlsx')
        else:
            angle = xlsxwriter.Workbook('StudentDataAngle.xlsx')
        worksheet_angle = angle.add_worksheet()
        if teacher_or_student == 'teacher':
            speed = xlsxwriter.Workbook('TeacherDataSpeed.xlsx')
        else:
            speed = xlsxwriter.Workbook('StudentDataSpeed.xlsx')
        worksheet_speed = speed.add_worksheet()
        if teacher_or_student == 'teacher':
            acceleration = xlsxwriter.Workbook('TeacherDataAcceleration.xlsx')
        else:
            acceleration = xlsxwriter.Workbook('StudentDataAcceleration.xlsx')
        worksheet_acceleration = acceleration.add_worksheet()
        if teacher_or_student == 'teacher':
            dataAnalyzes = xlsxwriter.Workbook('Data_Analyzes_Teacher.xlsx')
        else:
            dataAnalyzes = xlsxwriter.Workbook('Data_Analyzes_Student.xlsx')
        worksheet_dataAnalyzes = dataAnalyzes.add_worksheet()
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # Recolor image to RGB
            width, height = pyautogui.size()
            frame = cv2.resize(frame, (width, height))
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                position = angle_calculater(landmarks,mp_pose)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cap.release()
                    angle.close()
                    speed.close()
                    dataAnalyzes.close()
                    acceleration.close()
                    cv2.destroyAllWindows()
                    break
            except:
                pass

            if cu == 0:
                save_time[cu + 4] = round(0.033 + save_time[cu], 3)  # end - start

                for i in range(0, len(position.pose_data()) + 1):
                    if i != 20:
                        dn_angle.append([0, 0, 0, 0, 0, 0, 0, 0, 0, round(position.pose_data()[i], 3)])
                        save_loc.append([0, 0, 0, 0, 0])
                        worksheet_angle.write(row, i, 0)
                        worksheet_speed.write(row, i, str(0))
                        worksheet_acceleration.write(row, i, str(0))
                        save_data.append([0, None, 0])
                        movement_data.append([0, 0, 0, 0, 0])
                    dn_speed.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
                    speed_save.append([0, 0, 0, 0, 0])
                    acceleration_save.append([0, 0, 0, 0, 0])

                worksheet_angle.write(row, i, save_time[4])
                cu = cu + 1

            else:
                save_time = time_save(save_time[1], save_time[2], save_time[3], save_time[4])
                for i in range(0, len(position.pose_data())):
                    shift_register(dn_angle[i], round(position.pose_data()[i], 3))
                    model = savgol_filter(dn_angle[i], 9, 2)
                    save_loc[i] = loc_save(save_loc[i][1], save_loc[i][2], save_loc[i][3], save_loc[i][4],
                                           model[5], 0)
                    current_speed = speed_checker(save_loc[i][3], save_loc[i][4], save_time[2], save_time[3])

                    shift_register_speed(dn_speed[i], current_speed)
                    model1 = savgol_filter(dn_speed[i], 9, 2)
                    speed_save[i] = sava_speed(speed_save[i][1], speed_save[i][2], speed_save[i][3],
                                               speed_save[i][4], save_time[1], model1[5], 2)


                    if i + 1 == 20:
                        speed_save[i + 1] = sava_speed(speed_save[i + 1][1], speed_save[i + 1][2], speed_save[i + 1][3],
                                                       speed_save[i + 1][4],
                                                       round((save_time[2] + save_time[3]) / 2, 3),
                                                       current_speed, 0)
                        for j in range(0, len(position.pose_data())):

                            current_acceleration = acceleration_checker(speed_save[j][2], speed_save[j][3],
                                                                        speed_save[20][2], speed_save[20][3])
                            if j + 1 == 20:
                                acceleration_save[20] = sava_acceleration(acceleration_save[20],
                                                                          round((speed_save[20][1] + speed_save[20][
                                                                              0]) / 2, 3), current_acceleration, 0)

                            acceleration_save[j] = sava_acceleration(acceleration_save[j],
                                                                     round((speed_save[20][1] + speed_save[20][0]) / 2,
                                                                           3), current_acceleration, 2)
                            worksheet_acceleration.write(row, j, str(acceleration_save[j][3]))

                    worksheet_angle.write(row, i, str(save_loc[i][3]))
                    worksheet_speed.write(row, i, str(speed_save[i][3]))
                worksheet_angle.write(row, i + 1, save_time[3])
                worksheet_speed.write(row, i + 1, speed_save[20][3])
                worksheet_acceleration.write(row, i + 1, acceleration_save[20][3])

            row = row + 1
            stage_punch = pose_checker_punch(save_loc, movement_data,en_l_punch,en_r_punch)
            stage_kick = pose_checker_kick(save_loc, movement_data,en_r_kick,en_l_kick)
            if stage_kick =="left kick": en_l_kick = 1
            if stage_kick =="right kick": en_r_kick = 1
            if stage_punch =="left punch": en_l_punch = 1
            if stage_punch =="right punch": en_r_punch = 1


            for i in range(0, len(movement_data)):
                movement_data[i] = movment_data_cheker(i, save_loc[i], speed_save[i], acceleration_save[i],
                                                       movement_data[i])

                if movement_data[i][1] == "enter move" and save_data[i][0] == 0:
                    save_data[i][0] = acceleration_save[20][0]
                elif (stage_punch is not None) or (stage_kick is not None) :
                    if stage_punch == "right punch"  or  stage_punch=="right elbow" :
                        if save_data[1][0] != 0 and save_data[1][1] == None:
                            save_data[1][1] = stage_punch
                        if save_data[18][0] != 0 and save_data[18][1] == None:
                            save_data[18][1] = stage_punch


                    if stage_punch == "left punch" or stage_punch == "left punch pace forward" or \
                            stage_punch=="left elbow":

                        if save_data[19][0] != 0:
                            save_data[19][1] = stage_punch
                        if save_data[2][0] != 0:
                            save_data[2][1] = stage_punch

                    if stage_kick == "right kick" or "right knee":
                        if save_data[7][0] != 0:
                            save_data[7][1] = stage_kick
                        if save_data[9][0] != 0:
                            save_data[9][1] = stage_kick

                    if stage_kick == "left kick" or "left knee":
                        if save_data[6][0] != 0:
                            save_data[6][1] = stage_kick
                        if save_data[8][0] != 0:
                            save_data[8][1] = stage_kick

                if movement_data[i][1] == "finish" and save_data[i][2] == 0 and save_data[i][0] != 0  :
                    save_data[i][2] = save_time[4]

                    if save_data[i][1] is not None and save_data[i][2] != 0:
                        worksheet_dataAnalyzes.write(row_moves, 0, save_data[i][1])
                        worksheet_dataAnalyzes.write(row_moves, 1, save_data[i][0])
                        worksheet_dataAnalyzes.write(row_moves, 2, save_data[i][2])
                        worksheet_dataAnalyzes.write(row_moves, 3, i)
                        row_moves += 1
                        stage_punch =  None
                        stage_kick = None
                        save_data[i] = [0, None, 0]
                        en_r_kick = 0
                        en_l_kick = 0
                        en_l_punch = 0
                        en_r_punch = 0
            cv2.putText(image, stage_punch,
                        (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.putText(image, stage_kick,
                        (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2, cv2.LINE_AA)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                      )
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):



                cap.release()
                angle.close()
                speed.close()
                dataAnalyzes.close()
                acceleration.close()
                cv2.destroyAllWindows()
                break

    cap.release()
    angle.close()
    speed.close()
    dataAnalyzes.close()
    acceleration.close()
    cv2.destroyAllWindows()